In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [4]:
df.columns=df.columns.str.lower().str.replace(' ','_')
categorical_columns=list(df.dtypes[df.dtypes=='object'].index)

for c in categorical_columns:
    df[c]=df[c].str.lower().str.replace(' ','_')

In [5]:
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


In [6]:
tc=pd.to_numeric(df.totalcharges,errors='coerce')

In [7]:
df.totalcharges=pd.to_numeric(df.totalcharges,errors='coerce')

In [8]:
df.totalcharges=df.totalcharges.fillna(0)

In [9]:
df[tc.isnull()][['customerid','totalcharges']]

,customerid,totalcharges
488,4472-lvygi,0.0
753,3115-czmzd,0.0
936,5709-lvoeq,0.0
1082,4367-nuyao,0.0
1340,1371-dwpaz,0.0
3331,7644-omvmy,0.0
3826,3213-vvolg,0.0
4380,2520-sgtta,0.0
5218,2923-arzlg,0.0
6670,4075-wkniu,0.0


In [10]:
df.churn

0        no
1        no
2       yes
3        no
4       yes
       ... 
7038     no
7039     no
7040     no
7041    yes
7042     no
Name: churn, Length: 7043, dtype: object

In [11]:
df.churn=(df.churn=='yes').astype(int)

In [12]:
df_full_train,df_test=train_test_split(df,test_size=0.2,random_state=1)

In [13]:
df_train,df_val=train_test_split(df_full_train,test_size=0.25,random_state=1)

In [14]:
len(df_full_train),len(df_test),len(df_val)

(5634, 1409, 1409)

In [15]:
y_train=df_train.churn.values
y_test=df_test.churn.values
y_val=df_val.churn.values

In [16]:
del df_train['churn']
del df_val['churn']
del df_test['churn']

## Data Analysis

In [17]:
df_full_train.churn.value_counts(normalize=True)

0    0.730032
1    0.269968
Name: churn, dtype: float64

In [18]:
global_churn_rate=df_full_train.churn.mean()
round(global_churn_rate,2)

0.27

In [19]:
numerical=['tenure','monthlycharges','totalcharges']

In [20]:
df_full_train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [21]:
categorical=['gender', 'seniorcitizen', 'partner', 'dependents',
        'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']
      

In [22]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

### Churn rate

In [23]:
df_full_train.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
1814,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
5946,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
3881,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
2389,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
3676,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0


In [24]:
churn_female=df_full_train[df_full_train.gender=='female'].churn.mean()
churn_female

0.27682403433476394

In [25]:
churn_male=df_full_train[df_full_train.gender=='male'].churn.mean()
churn_male

0.2632135306553911

In [26]:
global_churn=df_full_train.churn.mean()
global_churn

0.26996805111821087

In [27]:
df_full_train.partner.value_counts()

no     2932
yes    2702
Name: partner, dtype: int64

In [28]:
churn_no_partner=df_full_train[df_full_train.partner=='no'].churn.mean()
churn_no_partner

0.3298090040927694

In [29]:
churn_partner=df_full_train[df_full_train.partner=='yes'].churn.mean()
churn_partner

0.20503330866025166

## Risk Ratio

In [30]:
churn_no_partner/global_churn
#>1 more likely to churn

1.2216593879412643

In [31]:
churn_partner/global_churn
#<1 less likely to churn

0.7594724924338315

In [32]:
from IPython.display import display

In [33]:
for c in categorical:
    print()
    print(c)
    df_group=df_full_train.groupby(c).churn.agg(['mean','count'])
    df_group['diff']=df_group['mean']-global_churn
    df_group['risk']=df_group['mean']/global_churn
    display(df_group)
    print()
    print('____________________________________________________')


gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980



____________________________________________________

seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208



____________________________________________________

partner


...


____________________________________________________

paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560



____________________________________________________

paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121



____________________________________________________


## Mutual Info

In [34]:
mutual_info_score(df_full_train.churn,df_full_train.contract)

0.0983203874041556

In [35]:
mutual_info_score(df_full_train.churn,df_full_train.gender)

0.0001174846211139946

In [36]:
mutual_info_score(df_full_train.churn,df_full_train.partner)

0.009967689095399745

In [37]:
def mutual_info_churn_score(series):
    return mutual_info_score(series,df_full_train.churn)

In [38]:
mi=df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

## Correlation

In [39]:
df_full_train.tenure.max()

72

In [40]:
df_full_train[numerical].corrwith(df_full_train.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

In [41]:
df_full_train[df_full_train.tenure<=2].churn.mean() 

0.5953420669577875

In [42]:
df_full_train[(df_full_train.tenure>2)&(df_full_train.tenure<=12)].churn.mean() 

0.3994413407821229

In [43]:
df_full_train[df_full_train.tenure>12].churn.mean() 

0.17634908339788277

In [44]:
df_full_train[df_full_train.monthlycharges <= 20].churn.mean()


0.08795411089866156

In [45]:
df_full_train[(df_full_train.monthlycharges > 20) & (df_full_train.monthlycharges <= 50)].churn.mean()


0.18340943683409436

In [46]:
df_full_train[df_full_train.monthlycharges > 50].churn.mean()


0.32499341585462205

## Encoding 

In [47]:
dv = DictVectorizer(sparse=False)


In [48]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)


In [49]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)


## Logistic regression

In [50]:
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_train,y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [51]:
model.coef_[0].round(3)

array([ 0.607, -0.058, -0.671, -0.028, -0.094,  0.052, -0.103, -0.071,
       -0.042, -0.08 , -0.376,  0.357, -0.103,  0.002, -0.248,  0.133,
       -0.008,  0.057, -0.103, -0.076,  0.217, -0.103, -0.236, -0.249,
        0.127, -0.116, -0.006, -0.101, -0.049,  0.067, -0.04 ,  0.133,
       -0.255,  0.199, -0.101, -0.103,  0.082, -0.064, -0.103,  0.045,
        0.17 , -0.103, -0.189, -0.067,  0.   ])

In [52]:
model.intercept_[0]

-0.1219348939703982

In [53]:
y_pred = model.predict_proba(X_val)[:, 1]


In [54]:
churn_decision=y_pred>=0.5

In [55]:
df_val[churn_decision].customerid

2504    8433-wxgna
4597    3440-jpscl
5591    7228-omtpn
4482    6711-fldfb
3610    2612-ranwt
           ...    
2611    5976-jcjrh
4211    2034-cgrhz
3999    5276-kqwhg
6240    6521-yytyi
5282    3049-solay
Name: customerid, Length: 312, dtype: object

In [56]:
(y_val==churn_decision).mean()

0.8055358410220014

In [57]:
df_pred=pd.DataFrame()
df_pred['probability']=y_pred
df_pred['prediction']=churn_decision.astype(int)
df_pred['actual']=y_val

In [58]:
df_pred['correct']=df_pred.prediction==df_pred.actual

In [59]:
df_pred

,probability,prediction,actual,correct
0,0.007359,0,0,True
1,0.209720,0,0,True
2,0.220707,0,0,True
3,0.575035,1,1,True
4,0.216670,0,0,True
...,...,...,...,...
1404,0.295302,0,0,True
1405,0.040019,0,1,False
1406,0.148879,0,0,True
1407,0.784839,1,1,True


## Model Interpretation

In [60]:
small=['contract','tenure','monthlycharges']

In [61]:
df_train[small].iloc[:10].to_dict(orient='records')

[{'contract': 'two_year', 'tenure': 72, 'monthlycharges': 115.5},
 {'contract': 'month-to-month', 'tenure': 10, 'monthlycharges': 95.25},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 75.55},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 80.85},
 {'contract': 'two_year', 'tenure': 18, 'monthlycharges': 20.1},
 {'contract': 'month-to-month', 'tenure': 4, 'monthlycharges': 30.5},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 75.1},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 70.3},
 {'contract': 'two_year', 'tenure': 72, 'monthlycharges': 19.75},
 {'contract': 'month-to-month', 'tenure': 6, 'monthlycharges': 109.9}]

In [62]:
dicts_train_small=df_train[small].to_dict(orient='records')
dicts_vall_small=df_train[small].to_dict(orient='records')

In [63]:
dv_small=DictVectorizer(sparse=False)
dv_small.fit(dicts_train_small)

DictVectorizer(sparse=False)

In [64]:
dv_small.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'monthlycharges', 'tenure'], dtype=object)

In [65]:
X_train_small=dv_small.transform(dicts_train_small)

In [66]:
model_small=LogisticRegression()
model_small.fit(X_train_small,y_train)

LogisticRegression()

In [67]:
w0=model_small.intercept_[0]
w0

-2.4767756620356254

In [68]:
w=model_small.coef_[0]
w.round(3)

array([ 0.97 , -0.025, -0.949,  0.027, -0.036])

In [69]:
dict(zip(dv_small.get_feature_names_out(), w.round(3)))


{'contract=month-to-month': 0.97,
 'contract=one_year': -0.025,
 'contract=two_year': -0.949,
 'monthlycharges': 0.027,
 'tenure': -0.036}

In [70]:
#bias + contract(monthly,1 year,2 years) +monthly charges+ tenure
#2 years only
-2.47 + (-0.949) + 30 * 0.027 + 24 * (-0.036)



-3.473

In [71]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


In [72]:
sigmoid(-3.473)

0.030090303318277657

## Using Model

In [73]:
dicts_full_train=df_full_train[categorical+numerical].to_dict(orient='records')


In [74]:
dicts_full_train[:3]

[{'gender': 'male',
  'seniorcitizen': 0,
  'partner': 'yes',
  'dependents': 'yes',
  'phoneservice': 'yes',
  'multiplelines': 'no',
  'internetservice': 'no',
  'onlinesecurity': 'no_internet_service',
  'onlinebackup': 'no_internet_service',
  'deviceprotection': 'no_internet_service',
  'techsupport': 'no_internet_service',
  'streamingtv': 'no_internet_service',
  'streamingmovies': 'no_internet_service',
  'contract': 'two_year',
  'paperlessbilling': 'no',
  'paymentmethod': 'mailed_check',
  'tenure': 12,
  'monthlycharges': 19.7,
  'totalcharges': 258.35},
 {'gender': 'female',
  'seniorcitizen': 0,
  'partner': 'no',
  'dependents': 'no',
  'phoneservice': 'yes',
  'multiplelines': 'no',
  'internetservice': 'dsl',
  'onlinesecurity': 'yes',
  'onlinebackup': 'yes',
  'deviceprotection': 'yes',
  'techsupport': 'yes',
  'streamingtv': 'no',
  'streamingmovies': 'yes',
  'contract': 'one_year',
  'paperlessbilling': 'no',
  'paymentmethod': 'credit_card_(automatic)',
  'tenur

In [75]:
dv=DictVectorizer(sparse=False)


In [76]:
X_full_train=dv.fit_transform(dicts_full_train)
y_full_train=df_full_train.churn.values

In [77]:
model = LogisticRegression().fit(X_full_train,y_full_train)

In [78]:
dicts_test=df_test[categorical+numerical].to_dict(orient='records')
X_test=dv.fit_transform(dicts_test)


In [79]:
y_pred=model.predict_proba(X_test)[:,1]

In [80]:
churn_decision=(y_pred>=0.5)

In [81]:
(churn_decision==y_test).mean()

0.815471965933286

In [82]:
customer=dicts_test[10]

In [83]:
X_small=dv.transform([customer])

In [84]:
X_small.shape

(1, 45)

In [86]:
model.predict_proba(X_small)

array([[0.5943189, 0.4056811]])